<a href="https://colab.research.google.com/github/alejandro56664-adl/unal-data-mining/blob/main/Tarea2_JheisonMorales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Minería de Datos - 2021-I
## Tarea 2: Preprocesamiento

![logo_unal](http://seeklogo.com/images/U/Universidad_Nacional_de_Colombia_-_Sede_Bogot_and__225_-logo-A05EAD6D0F-seeklogo.com.png)

Presentado por:
- [Jheison Alejandro Morales Vásquez](mailto:jhmoralesva@unal.edu.co)




In [ ]:
# Python
import pandas as pd 
import numpy as np
from numpy import mean, absolute 
import os
import math
import io

import matplotlib.pyplot as plt

from scipy.stats import trim_mean
from scipy.spatial import distance
from scipy.stats import entropy

from sklearn import preprocessing
from google.colab import files


## 1.Dado un conjunto de datos de una dimensión 



In [ ]:
X = np.array([-5.0, 23.0, 17.6, 7.23, 1.11])
print(X)

[-5.   23.   17.6   7.23  1.11]


 normalizar usando:
- Min_max normalizacion en el Intervalo [0,1]
- Min_max normalizacion en el Intervalo [-1,1]
- Desviación estandar
- Escala decimal en el intervalo [-1,1]
- Comparar los resultados y discutir ventajas y desventajas.

In [ ]:
#  Min_max normalizacion implementada
def norm_min_max(p_array, scale=(0,1)):
  p_array_min = np.amin(p_array)
  diff_max_min = np.amax(p_array) - p_array_min
  norm = (p_array - p_array_min)/diff_max_min
  return norm* (scale[1] - scale[0]) + scale[0]

In [ ]:
# 1. Min_max normalizacion en el Intervalo [0,1] implementada

X_norm_min_max_own = norm_min_max(X)
print(X_norm_min_max_own)


[0.         1.         0.80714286 0.43678571 0.21821429]


In [ ]:
# obtenido de: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html#sklearn.preprocessing.minmax_scale
from sklearn import preprocessing

# Se convierte en una matriz de 2 dimensiones donde (n_samples, n_features), en 
# esta caso 5 muestras y 1 caracteristica
X_2d = np.reshape(X, (-1, X.size)).T
print('Matriz 2d formato (n_samples, n_features):')
print(X_2d)

Matriz 2d formato (n_samples, n_features):
[[-5.  ]
 [23.  ]
 [17.6 ]
 [ 7.23]
 [ 1.11]]


In [ ]:
# 1. Min_max normalizacion en el Intervalo [0,1] scikit-learn
X_norm_min_max =  preprocessing.MinMaxScaler().fit_transform(X_2d)
print(X_norm_min_max)


In [ ]:
# 2. Min_max normalizacion en el Intervalo [-1,1] implementada
X_norm_min_max_own_2 = norm_min_max(X, scale=(-1,1))
print(X_norm_min_max_own_2)

In [ ]:
# 2. Min_max normalizacion en el Intervalo [-1,1] scikit-learn
X_norm_min_max_2 =  preprocessing.MinMaxScaler(feature_range=(-1,1)).fit_transform(X_2d)
print(X_norm_min_max_2)

In [ ]:
# Desviación estandar
# https://numpy.org/doc/stable/reference/generated/numpy.std.html
def norm_desviacion_estandar(p_array):
  return (p_array - p_array.mean())/p_array.std()


In [ ]:
norm_desviacion_estandar(X)

Para la normalización basado en la escala decimal:

- Encontrar el número mas grande en el conjunto de datos
- Contar el número de digitos _j_ del número mas grande.Count the number
- Dividir cada número por 10^j

In [ ]:
# Escala decimal en el intervalo [-1,1]
def norm_dec(p_array):
  # Encontrar el número mas grande en el conjunto de datos y
  max_value = p_array.max()
  # Contar el número de digitos j del número mas grande.Count the number
  j = np.ceil(np.log10(max_value))
  #Dividir cada número por 10^j
  norm = p_array/10**j
  return norm

In [ ]:

X_norm_dec = norm_dec(X)
print(X_norm_dec)

In [ ]:
#para llevarlo al rango -1,1 aplicamos min_max
X_norm_dec_min_max = norm_min_max(X_norm_dec, (-1,1))
print(X_norm_dec_min_max)

### Ventajas y desventajas

- Min_max normalizacion, es útil porque permite comprimir en un rango especifico los datos y reduce la suceptibilidad a las escalas.

- Desviación estandar: es útil para identificar objetos extraños (outliers) dentro de un conjunto de datos usando solamente los estadisticos.

- Escala decimal en el intervalo: Permite reducir de cierta manera la 'distancia' entre objetos cuyas escalas difieren mucho en orden de magnitud. Depende de la aplicación en donde esta distancia se pueda evitar.


## 2.Dado un conjunto de datos de 4 dimensiones con valores perdidos:

| I1 | I2 | I3 | I4 |
| -- | -- | -- | -- |
| 0  | 1  | 1  | 2  |
| 2  | 1  | ?  | 1  |
| 1  | ?  | ?  | 0  |
| ?  | 2  | 1  | ?  |
| 2  | 2  | 1  | 0  |


- Dado que el dominio para todos los atributos es [0,1,2] ¿Cuál debe ser el número de ejemplos “artificiales” si los valores perdidos son interpretados como “no importa el valor” y ellos son remplazados con todos los posibles valores para su dominio.


In [ ]:
df_perdidos = pd.DataFrame(np.array([[0,	1,	1,	2], 
                                      [2,	1,	'?',	1], 
                                      [1,	'?', '?',	0],
                                      ['?',	2,	1,	'?'],
                                      [2,	2,	1,	0]] ),
                            columns=['I1', 'I2', 'I3', 'I4'])

df_perdidos

In [ ]:
def generate_syntethic_rows(df, col, domain):
  df_new = pd.DataFrame()
  for index, row in df.iterrows():
    row_dict = row.to_dict()
    if row_dict[col] == '?':
      for d in domain: #dominio de los datos
        row_dict[col] = d
        df_new = df_new.append(row_dict, ignore_index=True)
    else:
      df_new = df_new.append(row_dict, ignore_index=True)
  return df_new 

In [ ]:
df_aux = df_perdidos
for col in df_perdidos:
  df_aux = generate_syntethic_rows(df_aux, col, ['0','1','2'])

df_aux

In [ ]:
print(df_aux.shape[0] - df_perdidos.shape[0])

R/: Sería necesario agregar 18

- La segunda instancia sería reemplazada por 3 nuevas instancias.
- La tercera y cuarta instancia por 9.

```nuevas_instancias = (3 - 1) + (3^2 - 1)*2 = 2 + 8*2 = 2 + 16 = 18```

- ¿Cuál otro método utilizaría para remplazar los valores pérdidos?

R/: Reemplazar cada valor perdido por la moda de la columna a la que pertence. 

## 3.El número de hijos de diferentes pacientes es dado por el siguiente vector:


In [ ]:
num_hijos = np.array([3,1,0,2,7,3,6,4,-2,0,0,10,15,6])

- Encontrar “outliers” usando parámetros estadisticos estándar: media y varianza
- Si el umbral cambia de ±3 desviaciones estandar a ±2 desviaciones estandar , ¿Cuál “outlier adicional” se encuentra?

In [ ]:
import statistics

mediana = statistics.median(num_hijos)
media = num_hijos.mean()
var   = num_hijos.var()
std   = num_hijos.std()


print('Media: ' + str(media))
print('Mediana: ' + str(mediana))
print('Varianza: ' + str(var))
print('Desviación: ' + str(std))

In [ ]:

def get_outliers(p_array, th):
  std_sample = p_array.std()/math.sqrt(len(p_array))
  z_p_array = (p_array - p_array.mean())/std_sample
  outliers = np.array([x for x in z_p_array if (x < -th).sum() or (x >= th).sum()])
  #devolvemos la transformación
  outliers = outliers*std_sample + p_array.mean()
  return outliers

In [ ]:
outliers = get_outliers(num_hijos, th=3)
outliers

In [ ]:
outliers = get_outliers(num_hijos, th=2)
outliers

Con 3 sigmas es suficiente para detectar valores atipicos, con dos parece que se incluyen valores que pueden ser considerados normales (como 1, por ejemplo)

## 4.Dado un conjunto de tres dimensiones



a) Describir el procedimiento e interpretar los resultados de detección de outliers basado en la media y varianza

In [ ]:
vector_3d = np.array([[1,2,0],[3,1,4],[2,1,5],[0,1,6],[2,4,3],[4,4,2],[5,2,1],[7,7,7],[0,0,0],[3,3,3]])
print(vector_3d)

In [ ]:
mediana = [statistics.median(vector_3d[:,0]), 
           statistics.median(vector_3d[:,1]), 
           statistics.median(vector_3d[:,2])]
           
media = vector_3d.mean(axis=0)
var   = vector_3d.var(axis=0)
std   = vector_3d.std(axis=0)

print('Mediana: ' + str(mediana))
print('Media: ' + str(media))
print('Varianza: ' + str(var))
print('Desviación: ' + str(std))

In [ ]:
outliers = get_outliers(vector_3d, 2)
outliers

In [ ]:
outliers = get_outliers(vector_3d, 3)
outliers

### interpretación, procedimiento comentario

Para descartar outliers se puede usar el [z-score](https://en.wikipedia.org/wiki/Standard_score), por ejemplo descartar aquellos que tengan 

```Z>=3 && Z<-3```


Explicación z-score
![diagrama explicativo](https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/The_Normal_Distribution.svg/2560px-The_Normal_Distribution.svg.png "Distribución normal")

Para el análisis se tuvieron en cuenta todas las columnas, suponiendo que el dominio de los datos es igual para todas las caracteristicas.

## 5.En Weka cargar el conjunto de datos iris.


En este caso se realizó el mismo procedimiento usando python.

- Eliminar manualmente valores (15%) en sus atributos, para simular valores perdidos. Luego aplicar varios métodos que estan en weka para remplazar esos valores perdidos. Discutir las diferencias entre el valor real y el que valor que lo remplaza, y las diferencias entre los métodos.
- Normalizar usando varios metodos.
- Discretizar usando varios métodos

### 5.1 Eliminar manualmente valores (15%) en sus atributos, para simular valores perdidos. 

In [ ]:
#carga de los datos de iris
df_iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', names=['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])
df_iris.head()

In [ ]:
# eliminación del 15% de los datos

# calculamos todos los valores en el dataset
def calculate_data_to_delete(df, percent):
  rows,cols = df.shape
  total_data = rows*cols #no tenemos en cuenta la clase
  return math.floor(total_data*percent)

In [ ]:
import random
def remove_data_in_row(p_row, max_to_delete):
  count_data_deleted = 0
  row = p_row.copy()
  for k,v in row.items():
    if random.uniform(0,1) > 0.5:
      row[k] = float('NaN')
      count_data_deleted = count_data_deleted +1
      if count_data_deleted >= max_to_delete:
        break
  return row,count_data_deleted


In [ ]:
row_test_dirty,_ = remove_data_in_row({'c1': 1.0, 'c2': 2.0, 'c3': 3.0, 'c4': 4.0}, max_to_delete=2)
row_test_dirty

In [ ]:
def remove_n_values(df, n):
  df_new = pd.DataFrame()
  max_to_delete_per_row = 2
  count_data_deleted = 0
  for index, row in df.iterrows():
    row_dirty = row.to_dict()
    #si todavia podemos quitar datos y por azar la siguiente columna la vamos ensuciar
    if count_data_deleted <= n and random.uniform(0,1) > 0.5:
      row_dirty, data_deleted = remove_data_in_row(row_dirty, max_to_delete_per_row)
      count_data_deleted = count_data_deleted + data_deleted
    df_new = df_new.append(row_dirty, ignore_index=True)
  return df_new


In [ ]:
df_test = remove_n_values(df_iris.iloc[1:5,:-1], 4 )
df_test

In [ ]:
def remove_values(df, percent):
  total_data_to_delete = calculate_data_to_delete(df, percent)
  return remove_n_values(df, total_data_to_delete)

In [ ]:
df_iris_missing_values = remove_values(df_iris.iloc[:,:-1], percent=0.15 )
df_iris_missing_values

In [ ]:
df_iris_missing_values.isnull().sum().sum()

### 5.2 Remplazo valores perdidos.

- Reemplazar por la media
- Reemplazar por la mediana

- Calcular error medio entre los conjuntos de datos original y con valores perdidos.

In [ ]:
# reemplazamos los valores por la media:
df_iris_missing_values_filled_media = df_iris_missing_values.fillna(df_iris_missing_values.median())
df_iris_missing_values_filled_media

In [ ]:
#reemplazamos por la mediana de cada columna
df_iris_missing_values_aux = df_iris_missing_values.dropna()

dic_aux = dict(zip(list(df_iris_missing_values.columns), 
                        [statistics.median(df_iris_missing_values_aux.iloc[:,1]), 
                          statistics.median(df_iris_missing_values_aux.iloc[:,1]), 
                          statistics.median(df_iris_missing_values_aux.iloc[:,2]),
                          statistics.median(df_iris_missing_values_aux.iloc[:,3])]))

dic_aux



In [ ]:
df_iris_missing_values_filled_median = df_iris_missing_values.fillna(dic_aux)
df_iris_missing_values_filled_median

In [ ]:
# calculamos el error
df_err_iris_filled_median = (df_iris.iloc[:,:-1] - df_iris_missing_values_filled_median).abs().mean()
df_err_iris_filled_median

In [ ]:
# calculamos el error
df_err_iris_filled_media = (df_iris.iloc[:,:-1] - df_iris_missing_values_filled_media).abs().mean()
df_err_iris_filled_media

TODO Comentario:
Aunque la diferencia es pequeña se percibe un menor error unado la media.

### 5.3 Normalizar los datos usando varios metodos

- min_max
- decimal
- Desviación estandar

In [ ]:
#min_max

df_iris_min_max =  preprocessing.MinMaxScaler().fit_transform(df_iris.iloc[:,:-1])
df_iris_min_max

In [ ]:
df_iris_norm_std =  norm_desviacion_estandar(df_iris.iloc[:,:-1])
df_iris_norm_std

In [ ]:
df_iris_norm_dec =  norm_dec(df_iris.iloc[:,:-1])
df_iris_norm_dec

TODO discusión de los resultados.

## 6.Realizar reducción de valores basado en la técnica de BIN con el mejor corte


Dado el conjunto de datos

In [ ]:
df_bin = pd.DataFrame(np.array([[1, 5.9, 3.4], 
                                [2, 2.1, 6.2], 
                                [1, 1.6, 2.8],
                                [2, 6.8, 5.8],
                                [1, 3.1, 3.1],
                                [1, 8.3, 4.1],
                                [2, 2.4, 5.0]]),
                            columns=['I1', 'I2', 'I3'])

df_bin

- Dimensión I2 usando la media como representantes de 2 BINS
- Dimensión I3 usando el limite más cercano como representante de 2 BINS


In [ ]:
def init_list_bin_pos(serie, n_bins):
  len = serie.shape[0]
  bin_width = math.floor(len/n_bins)
  result = []
  value = 0
  for x in range(0,n_bins):
    value = value + bin_width
    if x == n_bins - 1:
      value = len
    result.append(value)
  return result

In [ ]:
#prueba init_list_bin_pos
print(init_list_bin_pos(pd.Series([1,1,2,  3,3,4]), 2)) #[3, 6]
print(init_list_bin_pos(pd.Series([1,1,2,  3,4,5, 5,6,8,9]), 3)) #[3, 6, 10]
print(init_list_bin_pos(pd.Series([1,1, 2,3, 4,5,6]), 3)) #[2, 4, 7]
print(init_list_bin_pos(pd.Series([1,1, 2,3, 4,5,5,6]), 3)) #[2, 4, 8]
print(init_list_bin_pos(df_bin['I2'].sort_values(ignore_index=True), 2))

In [ ]:
def get_bin_pos(list_pos, i):
  limit_s = list_pos[i]
  if i>0:
    limit_i = list_pos[i-1]
  else:
    limit_i = 0
  return (limit_i, limit_s)

In [ ]:
print(get_bin_pos([4, 8, 10], i=0))
print(get_bin_pos([4, 8, 10], i=1))
print(get_bin_pos([4, 8, 10], i=2))

In [ ]:
def get_bin_width(list_pos, i):
  pos = get_bin_pos(list_pos, i)
  return pos[1] - pos[0]

In [ ]:
print(get_bin_width([4, 8, 10], i=0))
print(get_bin_width([4, 8, 10], i=1))
print(get_bin_width([4, 8, 10], i=2))

In [ ]:
def get_bin_items(serie, list_pos, i):
  pos = get_bin_pos(list_pos, i)
  return serie.iloc[pos[0]:pos[1]]

In [ ]:
#prueba get_bin_items
print(get_bin_items(pd.DataFrame([1,1,2,  3,3,4]), list_pos=[3,6], i=0))
print('----')
print(get_bin_items(pd.DataFrame([1,1,2,3, 4,5,5,6, 8,9]), list_pos=[4, 8, 10], i=1))
print('----')
print(get_bin_items(pd.DataFrame([1,1,2,3, 4,5,5,6, 8,9]), list_pos=[4, 8, 10], i=2))
print('----')
print(get_bin_items(df_bin['I2'].sort_values(ignore_index=True), list_pos=[3, 7], i=0))


In [ ]:
def move_items(serie, list_pos):
  #se mueven items apartir del bin1 hacia el bin anterior
  #si el item esta repetido, se mueven tantos como sea necesario
  #Ej: {1,1,2,  3,3,4} -> {1,1,2,3,3,  4}:
  new_list_pos = list_pos.copy()
  len_serie = serie.shape[0] -1
  last_bin = len(list_pos)-1
  
  for bin in range(0, last_bin):
    #el último bin no mueve datos
    if bin == last_bin:
      break

    limit_s = list_pos[bin]
    #para garantizar que se ejecute al menos una vez
    cont = True
    while cont:
      
      limit_s_value_before = serie.iloc[limit_s]
      if limit_s >= len_serie:
          break
      #movemos el limite una unidad hacia la derecha
      limit_s = limit_s + 1
      limit_s_value = serie.iloc[limit_s]
      #print(f'En limite: {limit_s}, esta el valor: {limit_s_value}, el anterior {limit_s_value_before}')

      cont = limit_s_value == limit_s_value_before
      
    new_list_pos[bin] = limit_s
  return new_list_pos
   

In [ ]:
print(move_items(df_bin['I2'].sort_values(ignore_index=True), list_pos=[3,7])) 

In [ ]:
# prueba move_items:
print(move_items(pd.Series([1,1,2,  3,3,4]), list_pos=[3,6])) #> [5,6]
print(move_items(pd.Series([1,1,2,  3,4, 5,5,6]), list_pos=[3,5,9])) #> [4,7,9]
print(move_items(pd.Series([1,1,2,  3,4, 5,6,6]), list_pos=[3,5,9])) #> [4,6,9]
print(move_items(pd.Series([1,1,2,  3,4,5, 6]), list_pos=[3,6,7])) #> [4,6,7]


In [ ]:
'''
serie: pandas Series
list_pos: listado de posiciones
fn_rep: función que calcula la representación del bin, recibe una serie que con los datos del bin
'''
def get_repr(serie, list_pos, fn_rep):
  n_bins = len(list_pos)
  l = list_pos[n_bins-1]
  result = []
  for i in range(0, n_bins):
    value = fn_rep(get_bin_items(serie, list_pos, i))
    result = result + [value]*get_bin_width(list_pos, i)
  return pd.Series(result)

In [ ]:
print(get_repr(serie=pd.Series([1,1,2,  3,4]), list_pos=[3,5], fn_rep=lambda serie: serie.mode().iloc[0])) # > [1 1 1 3 3 ]


In [ ]:
def compute_error(serie, list_pos, fn_rep):
  bin_repr = get_repr(serie, list_pos, fn_rep)
  er = (serie - bin_repr).abs().sum()
  return er,bin_repr

In [ ]:
def serie_discretization(serie, fn_rep, n_bins=2):

  # 1. Ordenar valores
  serie_o = serie.sort_values(ignore_index=True)
  #print(serie_o)

  # 2. Asignar aproximadamente igual número de valores (V_i) a cada bin
  # generamos la lista de limites superiores (indices) de los bins
  bin_list_pos = init_list_bin_pos(serie_o, n_bins) 

  # 2.1. Calcular error
  er,bin_repr = compute_error(serie_o, bin_list_pos, fn_rep)

  cont = True
  while cont:
    #3. Mover al borde el elemento V_i al siguiente o previo
    bin_list_pos_new = move_items(serie_o, bin_list_pos)

    #3.1 Calcular error
    er_new,bin_repr_new = compute_error(serie_o, bin_list_pos_new, fn_rep)
    
    if er_new < er:
      #print(f'ER nuevo: {er_new} ER viejo: {er} continuamos ...')
      bin_list_pos = bin_list_pos_new
      er = er_new
      bin_repr = bin_repr_new
    else:
      #print(f'ER nuevo: {er_new} ER viejo: {er} nos quedamos con el old.')
      cont = False
  return bin_repr, bin_list_pos, er

In [ ]:
 #de la moda se selecciona el primero con iloc[0]
serie_discretization(pd.Series([5, 1, 8, 2, 2, 2, 1, 8, 6, 9]),lambda bin: bin.mode().iloc[0],  3)

In [ ]:
serie_discretization(df_bin['I2'], lambda b: b.median()) #media

In [ ]:
serie_discretization(df_bin['I2'], lambda b: b.iloc[0]) #limite mas cercano

In [ ]:
serie_discretization(df_bin['I3'], lambda b: b.median()) #media

In [ ]:
serie_discretization(df_bin['I3'], lambda b: b.iloc[0]) #limite mas cercano

## 7.Hacer el ranking de las dimensiones realizando comparación de medias y varianzas
Dado el conjunto de datos:

In [ ]:
df_ranking = pd.DataFrame(np.array([[2.5, 1.6, 5.9, 0], 
                                    [7.2, 4.3, 2.1, 1], 
                                    [3.4, 5.8, 1.6, 1],
                                    [5.6, 3.6, 6.8, 0],
                                    [4.8, 7.2, 3.1, 1],
                                    [8.1, 4.9, 8.3, 0],
                                    [6.3, 4.8, 2.4, 1]]),
                                columns=['I1', 'I2', 'I3', 'C'])

df_ranking

In [ ]:
df_ranking.var()

In [ ]:
df_ranking_C_0 = df_ranking.query('C == 0')

In [ ]:
df_ranking_C_1 = df_ranking.query('C == 1')

In [ ]:
df_ranking_C_1.mean()

In [ ]:
'''
A, B: pandas Series
'''
def compute_var_rank(A, B):
  n1 = A.shape[0]
  n2 = B.shape[0]

  meanA = A.mean() 
  meanB = B.mean()
  
  SE = np.sqrt(A.var()/n1 + B.var()/n2)
  return np.abs(meanA - meanB)/SE

In [ ]:
def ranking(df_C0, df_C1, b):
  selected = {}
  ignored = {}
  for col in list(df_C0.columns):
    rank = compute_var_rank(df_C0[[col]], df_C1[[col]])
    rank_value = rank.iloc[0]
    if rank_value > b:
      selected[col] = rank_value
    else:
      ignored[col] = rank_value

  return selected, ignored

In [ ]:
compute_var_rank(df_ranking_C_0[['I1']], df_ranking_C_1[['I1']])

In [ ]:
# para hacer el ranking de los datos
ranking(df_ranking_C_0, df_ranking_C_1, 0.5)

## 8.Dado el conjunto de datos X, donde X1 y X2 son dimensiones numericas, X3 y X4 son dimensiones con datos categoricos

In [ ]:
df_hetero = pd.DataFrame(np.array([[2.7, 3.4, 1, 'A'], 
                                    [3.1, 6.2, 2, 'A'], 
                                    [4.5, 2.8, 1, 'B'],
                                    [5.3, 5.8, 2, 'B'],
                                    [6.6, 3.1, 1, 'A'],
                                    [5.0, 4.1, 2, 'B']]),
                                columns=['X1', 'X2', 'X3', 'X4'])

df_hetero

In [ ]:
df_hetero['X1'] = pd.to_numeric(df_hetero['X1'])
df_hetero['X2'] = pd.to_numeric(df_hetero['X2'])

In [ ]:
cols_groups = { 'cat': ['X3', 'X4'], 'num': ['X1', 'X2']}
cols_groups

### 8.1 Implementación imperativa inicial (poco eficiente)

In [ ]:
def make_pair_df(X, N, i):
  j = i+1
  #Se replica la fila actual
  X_i_a = [X.iloc[i,:].to_numpy()]*(N-j)
  
  X_j_a = X.iloc[j:,:].to_numpy()

  # Paso adicional para resetear los indices
  X_i = pd.DataFrame(X_i_a, columns=X.columns)
  X_j = pd.DataFrame(X_j_a, columns=X.columns)
  return X_i, X_j

In [ ]:
make_pair_df(df_hetero, df_hetero.shape[0], 0)

In [ ]:
#util para manejar columnas
def add_sufix(p_list, suffix):
  return list(map(lambda i: i+'_'+suffix, p_list))

In [ ]:
#util para manejar columnas
def build_column_names(p_list):
  return (add_sufix(p_list, '1'), add_sufix(p_list, '2'))

In [ ]:
#util para la reducción de dimensiones conocer
#a que grupo pertence (Categórico/Numérico)
def intersect_cols(cols1, cols2):
  return [value for value in cols1 if value in cols2]

In [ ]:
cols_u = ['X1','X2', 'X3']
cols_cat = ['X3', 'X4']
intersect_cols(cols_u, cols_cat) # > ['X3']

In [ ]:
def custom_merge(df1, df2):
  #closure util para mover columnas
  def select_and_rename(df_from, original_names, new_names):
    return df_from.loc[:,original_names].rename(columns=dict(zip(original_names,new_names)))

  #juntamos los objetos en el mismo dataframe
  cols = df1.columns.to_list()
  cols1,cols2 = build_column_names(cols)

  df_merged = select_and_rename(df1, cols, cols1)
  df_merged.loc[:,cols2] = select_and_rename(df2, cols, cols2)

  return df_merged
  

In [ ]:
df1, df2 = make_pair_df(df_hetero, df_hetero.shape[0], 0)
df_merged = custom_merge(df1, df2)
df_merged

In [ ]:
def calc_similarity_categoric(row, col1, col2):
  l = len(col1)
  if l == 0:
    return 0
  acc = 0.
  for i in range(0,l):
    #Para los categóricos simplemente se compararán
    acc = acc + (row[col1[i]] == row[col2[i]])
  return acc/l
    

In [ ]:
print(calc_similarity_categoric(df_merged.iloc[0,:], [],  [])) #> 0
print(calc_similarity_categoric(df_merged.iloc[0,:], ['X3_1'],  ['X3_2'])) #> 0
print(calc_similarity_categoric(df_merged.iloc[0,:], ['X3_1', 'X4_1'],  ['X3_2', 'X4_2'])) #> 0.5

In [ ]:
def calc_similarity_numeric(row, col1, col2):
  #Para los campos númericos se usara coseno
  if not col1 or not col2:
    return 0
  X = row[col1]
  Y = row[col2] 
  normXY = np.linalg.norm(X.values)*np.linalg.norm(Y.values)
  return  1. - np.dot(X,Y)/normXY


In [ ]:
print(calc_similarity_numeric(df_merged.iloc[0,:], [], [])) # 0
print(calc_similarity_numeric(df_merged.iloc[0,:], ['X1_1'], ['X1_2']))
print(calc_similarity_numeric(df_merged.iloc[0,:], ['X1_1', 'X2_1'], ['X1_2', 'X2_2'])) #0.021

In [ ]:

def build_similarity_by_row(cols, cols_groups):

  cnum1,cnum2 = build_column_names(intersect_cols(cols, cols_groups['num']))
  ccat1,ccat2 = build_column_names(intersect_cols(cols, cols_groups['cat']))

  def f(row):
    s_num = calc_similarity_numeric(row, cnum1, cnum2)
    s_cat = calc_similarity_categoric(row, ccat1,  ccat2)
    s = (s_num + s_cat)/2
    return s
  
  return f

In [ ]:
calc_similarity_by_row = build_similarity_by_row(list(df_merged.columns), cols_groups=cols_groups)
calc_similarity_by_row(df_merged.iloc[0,:])

In [ ]:
def build_similarity_vector(X, cols_groups):
  N = X.shape[0]
  s_aux = []

  #se crea closure para aplicar en cada row
  calc_similarity_by_row = build_similarity_by_row(list(X.columns), cols_groups)

  # se crea el vector de similaridades
  for i in range(0,N-1):
    X_i, X_j = make_pair_df(X, N, i)
    s_aux.append(custom_merge(X_i, X_j).apply(calc_similarity_by_row, axis=1))
  
  return pd.concat(s_aux)

In [ ]:
S_ij = build_similarity_vector(df_hetero[['X3','X4', 'X2']], cols_groups=cols_groups)
S_ij

In [ ]:
#Este método tiene muy bajo rendimiento, mejorar
def calc_entropy(X, cols_groups):
  S_ij = build_similarity_vector(X, cols_groups)
  S_ij_ = 1 - S_ij

  # se calcula la entropía
  return -(S_ij*np.log(S_ij) + S_ij_*np.log(S_ij_)).sum()

In [ ]:
calc_entropy(df_hetero[['X3','X4', 'X2']], cols_groups)

### 8.2 Implementación usando SciPy

In [ ]:
# función de utilidad para convertir datos categoricos a númericos
def cat2num(df, colnames):
  dict_le = {}
  result = pd.DataFrame()
  for col in colnames:
    dict_le[col] = preprocessing.LabelEncoder()
    dict_le[col].fit(df[col])
    result[col] = dict_le[col].transform(df[col])
  return result, dict_le

In [ ]:
#función que calcula la similitud
# para los datos númericos se usa coseno y para los categóricos se una Hamming
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

def calc_entropy2(df, cols_groups):

  # obtenemos las columnas del dataframe
  cols = df.columns
  rows = df.shape[0]

  #closure útilidad
  def intersect_cols(cols1, cols2):
    return [value for value in cols1 if value in cols2]

  #closure útilidad
  def df_or_zeros(p_df, fn):
    if p_df.shape[1] > 0:
      return fn(p_df)
    else: 
      return np.zeros(shape=(rows, rows))

  # seleccionamos las columnas númericas
  cols_num = intersect_cols(cols, cols_groups['num'])
  # seleccionamos las columnas categóricas
  cols_cat = intersect_cols(cols, cols_groups['cat'])

  df_num = df[cols_num]
  #df_num = df_or_zeros(df, cols_num, lambda df, c: df[c]) 

  df_cat,_ = cat2num(df, cols_cat)
  #df_cat = df_or_zeros(df, cols_cat, lambda df, c: cat2num(df, c)[0])

  #calculamos las distancias por separado
  d_num = df_or_zeros(df_num, lambda d: distance.cdist(d, d, 'cosine'))
  #print(d_num)

  d_cat = df_or_zeros(df_cat, lambda d: distance.cdist(d, d, 'hamming'))
  #print(d_cat)
  #se calcula el factor para el promedio ponderado
  factor = np.array([len(cols_num), len(cols_cat)])/len(cols)

  # ponderado entre los campos númericos y categóricos
  dis = (factor[0]*d_num + factor[1]*d_cat)/2
  #print(dis)

  # transformamos a similaridad
  sim = 1. - dis
  sim_ = dis # 1 - sim, no lo calculamos otra vez

  # se calcula la entropía
  e = (entropy(sim) + entropy(sim_)).sum()
  return e

In [ ]:
calc_entropy2(df_hetero[['X4', 'X3']], cols_groups )


- Aplicar el método se selección de características basado en la entropía para reducir una dimensión (mostrar pasos).
- Implementar un programa para realizar el “ranking” de dimensiones usando entropía.

In [ ]:
# Implementación algoritmo:

# 1. Tomamos todo el conjunto de datos
F = df_hetero
ranking = []

cont = True
while cont:
  # 2. Calculamos la entropía
  E_F = calc_entropy2(F, cols_groups) #calc_entropy(F, cols_groups)

  # 3. Por cada dimensión (Columna)
  cols = list(F.columns)
  #condición de parada: hasta que solo quede una dimensión
  if len(cols) == 1:
    ranking.append(cols[0])
    break
  
  E_F_list = []
  E_F_dict = {}

  for col in cols:

    #eliminamos el atributo
    cols_aux = cols.copy()
    cols_aux.remove(col)
    Ff = F[cols_aux]
    
    E_Ff = calc_entropy2(Ff, cols_groups)  #calc_entropy(F, cols_groups)
    
    diff = E_F - E_Ff

    E_F_list.append(diff)
    E_F_dict[diff] = col

  # luego de calcular todos las entropias, buscamos la diferencia minima
  min_E_F = min(E_F_list)
  col = E_F_dict[min_E_F]

  cols.remove(col)
  F = F[cols]
  ranking.append(col)

  print(E_F_dict)
ranking

## 9.Al conjunto de datos Adult del repositorio de Machine learning:



Este conjunto de datos corresponde a datos del censo de USA de 1994 (https://archive.ics.uci.edu/ml/index.php).

In [ ]:
# Carga de los datos
#uploaded = files.upload()
#df_adult = pd.read_csv(io.BytesIO(uploaded['adult.data']), sep=',', names=['age', 'workclass', 'fnlwgt', 'education','education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income' ])
df_adult = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', sep=',', names=['age', 'workclass', 'fnlwgt', 'education','education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income' ])

df_adult.shape
df_adult

### 9.1Convertir todos los atributos numéricos a categóricos utilizando dos estrategias diferentes.


In [ ]:
#seleccionamos las columnas que son numericas
cols_num = ['age', 'hours-per-week', 'education-num', 'fnlwgt', 'capital-gain', 'capital-loss' ]

In [ ]:
def serie_discretization_search_bins(serie, fn_rep, stop_diff=0.3, max_bins = 10):
   #TODO implementar un algoritmo de busqueda mas eficiente
  # calculamos los er
  
  n_bins = 2
  serie_disc, _, er = serie_discretization(serie, fn_rep, n_bins)
  diff = 1
  cont = True
  while cont:
    serie_disc_new, _, er_new = serie_discretization(serie, fn_rep, n_bins)
    diff = er_new/er
    print(f'diff er: {diff}')
    if diff <= stop_diff or n_bins >= max_bins:
      cont = False
      serie_disc = serie_disc_new
      er = er_new
    else:
      n_bins = n_bins +1

  print(f'mejor encontrado -> n_bins: {n_bins}, er: {er}')
  return serie_disc, er

In [ ]:
df_adult_num = df_adult[cols_num]

In [ ]:
serie_discretization_search_bins(df_adult_num['age'], lambda b: b.mean())

In [ ]:
def df_discretization(df, col_names, fn_rep, stop_diff=0.5, max_bins = 10):
  result = df
  for col in col_names:
    print(f'Se inicia proceso de discretización para la col: {col}')
    result = result.sort_values(by=[col], ignore_index=True)
    serie_disc, _ = serie_discretization_search_bins(result[col], fn_rep, stop_diff, max_bins )
    result[col] = serie_disc

  return result
  

In [ ]:
df_adult_disc = df_discretization(df_adult, ['age', 'hours-per-week', 'education-num', 'fnlwgt'] , lambda b: b.mean())

In [ ]:
df_adult_disc = df_discretization(df_adult_disc, ['capital-gain', 'capital-loss'] , lambda b: b.iloc[0],  max_bins = 2) #limite mas cercano

In [ ]:
df_adult_disc.hist()


### 9.2Transformar el conjunto de datos de manera que todos los atributos sean numéricos.

In [ ]:
#seleccionamos las columnas que son categóricas
cols_cat = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income' ]

df_adult['workclass']

In [ ]:
#Para transformar datos categóricos a númericos se usará
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
s = le.fit(df_adult['sex'])

In [ ]:
s.classes_

In [ ]:
le.transform(df_adult['sex'])

In [ ]:
def cat2num(df, colnames):
  dict_le = {}
  result = pd.DataFrame()
  for col in colnames:
    dict_le[col] = preprocessing.LabelEncoder()
    dict_le[col].fit(df[col])
    result[col] = dict_le[col].transform(df[col])
  return result, dict_le


In [ ]:
cat2num(df_adult, cols_cat)

## 10.Aplicación PCA




El conjunto de datos seleccionado es [Smartphone Dataset for Human Activity Recognition (HAR) in Ambient Assisted Living (AAL) Data Set
](https://archive.ics.uci.edu/ml/datasets/Smartphone+Dataset+for+Human+Activity+Recognition+%28HAR%29+in+Ambient+Assisted+Living+%28AAL%29) 


Descripción del conjunto de datos:

El conjunto contiene información de medidas de sensores para reconocer la posición de la actividad humana. El conjunto muestra los valores de los sensores por cada eje (X, Y, Z) y esta aumentado por variables calculadas como correlaciones entre ejes, entropía, energía etc llegando a un total de 561 columnas.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00364/dataset_uci.zip

In [ ]:
!unzip dataset_uci.zip

In [ ]:
HOME_DATA_SET = './dataset_uci'

In [ ]:
# Abrimos el features.txt para explorar su contenido
headers = [] 
with open(HOME_DATA_SET + '/features.txt', 'r') as header_txt:
    headers = [line.strip() for line in header_txt.readlines()]
headers 

#headers = all_data[0].split(',')

In [ ]:
har_aal_filename = HOME_DATA_SET + '/final_X_test.txt'
df_har_aal = pd.read_csv(har_aal_filename, ',', names=headers)

In [ ]:
df_har_aal.head()

**PCA**

Es un método clásico para reducir el número de atributos en los datos proyectando los datos desde su espacio original de alta dimensión en un espacio de menor dimensión. Los nuevos atributos (también conocidos como componentes) creados por PCA tienen las siguientes propiedades: 

- son combinaciones lineales del atributo original
- son ortogonales (perpendiculares) entre sí
- capturan la máxima cantidad de variación en los datos.

Importante:

_No es una técnica de selección de características_. Más bien, es una técnica de combinación de características. Porque cada PC es una combinación aditiva ponderada de todas las columnas del conjunto de datos original. Más sobre esto cuando lo implemente en la siguiente sección.

Para esta sección se utilizó el tutorial:

https://www.machinelearningplus.com/machine-learning/principal-components-analysis-pca-better-explained/

In [ ]:
from sklearn.decomposition import PCA
# PCA
pca = PCA()
df_pca = pca.fit_transform(X=df_har_aal)

# Store as dataframe and print
df_pca = pd.DataFrame(df_pca)
print(df_pca.shape)  #> (3147, 784)
df_pca.round(2).head()

Importante:

- El dataframe resultante contiene los componentes principales (PC) y tiene las mismas dimensiones del conjunto original.

- *pca.components_* este objeto contiene los pesos:
 - cada PC es el producto punto entre los pesos y los datos autoescalados.
 

In [ ]:
# para calcular cuantos PC son necesarios, teniendo en cuenta que 90% estaría bien
variance_exp_cumsum = pca.explained_variance_ratio_.cumsum().round(2)
max_pcs = variance_exp_cumsum[variance_exp_cumsum <= 0.9].shape[0]
max_pcs

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1,1,figsize=(16,7), dpi=100)
plt.plot(variance_exp_cumsum, color='firebrick')
plt.vlines(x = max_pcs, ymin = variance_exp_cumsum[0], ymax =1.0, colors = 'purple') 
plt.title('Varianza acumulada%', fontsize=18)
plt.xlabel('# de PCs', fontsize=12)
plt.show()

En conclusión aunque los PC no tienen una significado propio asociada con los datos, si que es una herramienta útil para reducir considerablemente las dimensiones, en este caso, se paso de 561 a solo 41, implica una reducción casi del 93%.

## 11.Análisis valores perdidos conjunto de datos Labour

En este punto se realizará un procedimiento similar al realizado en el punto 5.

Para evitar el procesamiento manual, se carga desde la fuente: https://raw.githubusercontent.com/renatopp/arff-datasets/master/classification/labor.arff



In [ ]:
!wget https://raw.githubusercontent.com/renatopp/arff-datasets/master/classification/labor.arff

In [ ]:
from scipy.io import arff
from io import StringIO

data, meta = arff.loadarff('labor.arff')

In [ ]:
df_labor = pd.DataFrame(data)

In [ ]:
df_labor

In [ ]:
# Calculamos el porcentaje de valores perdidos por atributo
df_labor_missing_percent = df_labor.isna().sum()/df_labor.shape[0]*100
df_labor_missing_percent[df_labor_missing_percent > 0]

Vamos a revisar lo que pasa con los datos de:


In [ ]:
def imprimir_estadisticos(df):
  df_dropna = df.dropna()
  print(f'Media:   {df_dropna.mean()}')
  print(f'Mediana: {df_dropna.median()}')
  print(f'std:     {df_dropna.std()}')
  print('---------')
  print(f'moda: {df_dropna.mode()}')


In [ ]:
# standby-pay:                  84.2% de datos faltantes
imprimir_estadisticos(df_labor[['standby-pay']])

In [ ]:
# wage-increase-third-year:     73.7%
imprimir_estadisticos(df_labor[['wage-increase-third-year']])

Para el caso de **standby-pay** y **wage-increase-third-year** y aunque depende del tipo de tarea a realizar con el conjunto de datos lo mas sensato sería quitarlos ya que al faltar mas del 70% de los datos no aportan información.  Tratar de reemplazarlos puede causar un riesgo de sesgo dentro del conjunto.  

Aúnque las medidas de centralidad **wage-increase-third-year** son mas o menos cercanas y la dispersión no es muy alta, aún consideraría quitarlo por la falta de datos.

In [ ]:
# shift-differential:           45.614035
imprimir_estadisticos(df_labor[['shift-differential']])

**shift-differential** tiene menos de la mitad de los datos y unas medidas de centralidad alejadas entre si ademas de una dispersión relativamente alta, por lo que en este caso se podria realizar una tarea de regresión a partir de los datos existentes, para estimar el 55% de datos faltantes.

Ahora se analizarán los atributos con menos del 20% de datos faltantes:

In [ ]:
# wage-increase-second-year    19.298246
imprimir_estadisticos(df_labor[['wage-increase-second-year']])

En este caso, la despersión es de un tamaño razonable y las medidas de centralidad son muy cercanas entre sí. 

Se revisa como cambían las estadisticas, reemplazando los valores faltantes por '4.0':

In [ ]:
df_labor_wage_increase_second_year = df_labor[['wage-increase-second-year']].fillna(4.0)
imprimir_estadisticos(df_labor_wage_increase_second_year)

In [ ]:
# working-hours                10.526316
imprimir_estadisticos(df_labor[['working-hours']])

En el caso de **working-hours** tenemos dos opciones, reemplazar por la mediana o la moda y escoger el que cambie en menor porcentaje los estadisticos:

In [ ]:
print('Usando la mediana: ')
imprimir_estadisticos(df_labor[['working-hours']].fillna(38.0))
print('Usando la moda: ')
imprimir_estadisticos(df_labor[['working-hours']].fillna(40.0))

En este caso es mejor usar la mediana = 38.0

In [ ]:
# statutory-holidays            7.017544
imprimir_estadisticos(df_labor[['statutory-holidays']])

Finalmente para **'statutory-holidays'** no hay mucha diferencia entre el uso de la media y la mediana, preferiblemente la mediana para evitar agregar datos atípicos.

In [ ]:
print('Usando la mediana: ')
imprimir_estadisticos(df_labor[['statutory-holidays']].fillna(11.0))
print('Usando la media: ')
imprimir_estadisticos(df_labor[['statutory-holidays']].fillna(11.09))